# read and interpolate limb and gravite-darkening coefficient of [Claret et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...634A..93C/abstract)

In [3]:
from astropy.io import fits
import numpy as np
from astropy.table import Table

In [10]:
from sdOB.utils.lyd import claret_tab, lydClaret

# limb darkening coefficient of Tess from LDCs DA models

In [5]:
fname = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2020/Gravity_limb-darkening/TABLE104C'
ctab = claret_tab()
names, data = ctab.read_tab2020(fname)
grid_pars = data[:, [0,1]].T
indc = [4+5*_ for _ in [0, 1, 2, 3, 6]]
grid_data = data[:, indc].T
axis_values, pixelgrid = ctab.create_pixeltypegrid(grid_pars=grid_pars, grid_data=grid_data)
p = np.array([[5.25], [36000]])
cc= ctab.interpolate(p, axis_values=None, pixelgrid=None)
a1, a2, a3, a4, bfac = cc.T[0]

a1, a2, a3, a4, bfac

(0.7649600000000001,
 -1.1051200000000008,
 1.0348600000000008,
 -0.37220000000000036,
 1.72796)

# Gravity darkening coeffiecient of Tess from LDCs DA models

In [6]:
fname = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2020/Gravity_limb-darkening/TABLE105C'
ctab = claret_tab()
names, data = ctab.read_tab2020(fname, skiprows=2, skipcolumns=3)
grid_pars = data[:, [0,1]].T
indc = [4+5*_ for _ in [0, 1]]
grid_data = data[:, indc].T
axis_values, pixelgrid = ctab.create_pixeltypegrid(grid_pars=grid_pars, grid_data=grid_data)
p = np.array([[5.25], [36000]])
cc= ctab.interpolate(p, axis_values=None, pixelgrid=None)
cc.T[0]

array([ 0.3829 , -0.01608])

In [11]:
##-------------------initialize class of Claret_tab DA------------------------------------------
_fname = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2020/Gravity_limb-darkening/TABLE104C'                                                               
ldtab = claret_tab()
names, data = ldtab.read_tab2020(_fname)
grid_pars = data[:, [0,1]].T
indc = [4+5*_ for _ in [0, 1, 2, 3, 6]]
grid_data = data[:, indc].T
_, _ = ldtab.create_pixeltypegrid(grid_pars=grid_pars, grid_data=grid_data)

_fname = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2020/Gravity_limb-darkening/TABLE105C'                                                               
# downloaded from https://cdsarc.cds.unistra.fr/ftp/J/A+A/634/A93/OriginalTab.tar.gz
ytab = claret_tab()
names, data = ytab.read_tab2020(_fname, skiprows=2, skipcolumns=3)
grid_pars = data[:, [0,1]].T
indc = [4+5*_ for _ in [0, 1]]
grid_data = data[:, indc].T                                                                                                                                               
_, _ = ytab.create_pixeltypegrid(grid_pars=grid_pars, grid_data=grid_data)

Teff, logg = 35030.0, 5.307

##-----------limb, gravity and doppler boosting factor--------------------------------   
lydinterp = lydClaret()
_ldc_bfac_y = lydinterp.interpolate_ldy_from_DA(Teff, logg, ldtab=ldtab, ytab=ytab)
ldc_1 = [_[0] for _ in _ldc_bfac_y[:4]]  # DA LTE paramerters =  np.array([[5.25], [36000]])                                                                             
gdc_1 =_ldc_bfac_y[4][0]# DA LTE
bfac_1 = _ldc_bfac_y[5]
####----------------------------------------------------------------------------------

# Limb darkening coeffiecient of Tess from LDCs ATLAS models xi =2

In [7]:
fname_ldc = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2017/J_A+A_600_A30/table28.dat.gz'

ctab = claret_tab()
_ = ctab.read_tab2017_ldc(fname_ldc)
_, _ = ctab.fix_parameter(index = 3, fix_value=2)
axis_values, pixelgrid = ctab.create_pixeltypegrid()
p = np.array([[5], [np.log10(34999)], [0]])
cc= ctab.interpolate(p, axis_values=axis_values, pixelgrid=pixelgrid)
cc.T[0]

array([ 0.88047901, -1.23483012,  1.09692598, -0.37327191])

# Gravity darkening coeffiecient of Tess from LDCs ATLAS models

In [29]:
fname_y = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2017/J_A+A_600_A30/table29.dat.gz'
ctab = claret_tab()
_ = ctab.read_tab2017_y(fname_y)
_, _ = ctab.fix_parameter(index = 3, fix_value=2)
axis_values, pixelgrid = ctab.create_pixeltypegrid()
p = np.array([[5], [np.log10(34090)], [0]])
cc= ctab.interpolate(p, axis_values=axis_values, pixelgrid=pixelgrid)
cc.T

array([[0.43510931]])

In [67]:
##-------------------initialize class of Claret_tab ATLAS------------------------------------------
_fname = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2017/J_A+A_600_A30/table28.dat.gz'
ldtab = claret_tab()
_ = ldtab.read_tab2017_ldc(_fname)
_, _ = ldtab.fix_parameter(index = 3, fix_value=2)
_, _ = ldtab.create_pixeltypegrid()


_fname = '/home/lijiao/lijiao/Documents/Claret_limb_gravity_beaming/2017/J_A+A_600_A30/table29.dat.gz'                                                             
# downloaded from https://cdsarc.cds.unistra.fr/ftp/J/A+A/634/A93/OriginalTab.tar.gz
ytab = claret_tab()
_ = ytab.read_tab2017_y(_fname)
_, _ = ytab.fix_parameter(index = 3, fix_value=2)                                                                                                                                             
_, _ = ytab.create_pixeltypegrid()

Teff, logg, Z = 34090, 5, 0

Teff, logg, Z = 30618, 3.8, np.log10(0.38)
Teff, logg, Z = 27000, 1, np.log10(0.38)

##-----------limb, gravity factor--------------------------------   
lydinterp = lydClaret()
_ldc_bfac_y = lydinterp.interpolate_ldy3(Teff, logg, Z, ldtab=ldtab, ytab=ytab)
ldc_1 = [_[0] for _ in _ldc_bfac_y[:4]]                                                                              
gdc_1 =_ldc_bfac_y[4][0][0]
print(f'a1,a2,a3,a4 = ', ldc_1)
print(f'y=', gdc_1)
####----------------------------------------------------------------------------------

a1,a2,a3,a4 =  [nan, nan, nan, nan]
y= nan


True

In [46]:
np.max(ytab.grid_pars[2])

1.0